In [ ]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor

# Load and clean data
df = pd.read_excel("CFL_DS2.xlsx")
df.columns = df.columns.str.strip()

# Define available quarters
quarter_targets = [q for q in ["FY24 Q3", "FY24 Q4", "FY25 Q1", "FY25 Q2"] if q in df.columns]

# Feature columns (exclude all targets and Product Name)
feature_cols = [col for col in df.columns if col not in ["Product Name"] + quarter_targets]

# Identify column types
numeric_cols = df[feature_cols].select_dtypes(include=["int64", "float64"]).columns.tolist()
categorical_cols = df[feature_cols].select_dtypes(include=["object"]).columns.tolist()

# Preprocessing
numeric_transformer = Pipeline(steps=[("imputer", SimpleImputer(strategy="median"))])
categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("encoder", OneHotEncoder(handle_unknown="ignore", sparse_output=False))
])
preprocessor = ColumnTransformer([
    ("num", numeric_transformer, numeric_cols),
    ("cat", categorical_transformer, categorical_cols)
])

# Models
models = {
    "Random Forest": RandomForestRegressor(n_estimators=100, random_state=42),
    "Linear Regression": LinearRegression(),
    "Gradient Boosting": GradientBoostingRegressor(n_estimators=100, random_state=42),
    "Decision Tree": DecisionTreeRegressor(random_state=42)
}

def evaluate_model(model, X_eval, y_eval):
    preds = model.predict(X_eval)
    bias = (preds - y_eval) / y_eval
    accuracy = np.maximum(0, 1 - np.abs(bias))
    return np.mean(bias), np.mean(accuracy)

# Model scoring
model_scores = []

for model_name, model in models.items():
    all_bias = []
    all_accuracy = []

    for i in range(len(quarter_targets) - 1):
        current_q = quarter_targets[i]
        next_q = quarter_targets[i + 1]

        train_mask = df[current_q].notna()
        X_train = df.loc[train_mask, feature_cols]
        y_train = df.loc[train_mask, current_q]

        eval_mask = df[next_q].notna()
        X_eval = df.loc[eval_mask, feature_cols]
        y_eval = df.loc[eval_mask, next_q]

        if len(X_train) == 0 or len(X_eval) == 0:
            continue

        pipeline = Pipeline([
            ("preprocessor", preprocessor),
            ("regressor", model)
        ])
        pipeline.fit(X_train, y_train)

        bias, accuracy = evaluate_model(pipeline, X_eval, y_eval)
        all_bias.append(bias)
        all_accuracy.append(accuracy)

    avg_bias = np.mean(all_bias)
    avg_accuracy = np.mean(all_accuracy)

    model_scores.append({
        "Model": model_name,
        "Avg Bias": avg_bias,
        "Avg Accuracy": avg_accuracy
    })

    print(f"Model: {model_name}")
    print(f"  Avg Bias: {avg_bias:.4f}")
    print(f"  Avg Accuracy: {avg_accuracy:.4f}\n")

# Select best model
best_model_info = max(model_scores, key=lambda x: x["Avg Accuracy"])
best_model_name = best_model_info["Model"]
best_model_obj = models[best_model_name]

print(f"Best model selected: {best_model_name}")

# Final training on FY25 Q1 → Predict FY25 Q2
train_mask = df["FY25 Q1"].notna()
X_train_final = df.loc[train_mask, feature_cols]
y_train_final = df.loc[train_mask, "FY25 Q1"]
X_forecast = df[feature_cols]

final_pipeline = Pipeline([
    ("preprocessor", preprocessor),
    ("regressor", best_model_obj)
])
final_pipeline.fit(X_train_final, y_train_final)

# Predict
y_pred_point = final_pipeline.predict(X_forecast)

# Add realistic randomness using bootstrapped residuals + stochastic drift ---
y_train_pred = final_pipeline.predict(X_train_final)
residuals = y_train_final - y_train_pred

# Bootstrapping from residuals (real-world-like noise)
n_simulations = 1000
simulated_forecasts = []

for _ in range(n_simulations):
    # 1. Bootstrapped residuals
    bootstrapped_residuals = np.random.choice(residuals, size=len(X_forecast), replace=True)

    # 2. Add mild stochastic drift (simulating market conditions)
    drift_factor = np.random.normal(loc=1.0, scale=0.02, size=len(X_forecast))  # small random upward/downward movement

    # 3. Combine base forecast + bootstrapped error + drift
    simulated = (y_pred_point + bootstrapped_residuals) * drift_factor
    simulated_forecasts.append(simulated)

simulated_forecasts = np.array(simulated_forecasts)
mean_forecast = simulated_forecasts.mean(axis=0)
lower_bound = np.percentile(simulated_forecasts, 5, axis=0)
upper_bound = np.percentile(simulated_forecasts, 95, axis=0)

# Save results
df["FY25 Q2 (Forecasted Mean)"] = mean_forecast
df["FY25 Q2 (Lower Bound 5%)"] = lower_bound
df["FY25 Q2 (Upper Bound 95%)"] = upper_bound

# Export output
df.to_excel("CFL_DS2_FY25Q2_Forecasted_Simulated.xlsx", index=False)
print("Final forecast with real-world randomness saved as 'CFL_DS2_FY25Q2_Forecasted_Simulated.xlsx'")

Model: Random Forest
  Avg Bias: 0.0142
  Avg Accuracy: 0.8259

Model: Linear Regression
  Avg Bias: 0.0113
  Avg Accuracy: 0.8430

Model: Gradient Boosting
  Avg Bias: 0.0114
  Avg Accuracy: 0.8430

Model: Decision Tree
  Avg Bias: 0.0113
  Avg Accuracy: 0.8430

✅ Best model selected: Linear Regression
📁 Final forecast with real-world randomness saved as 'CFL_DS2_FY25Q2_Forecasted_Simulated.xlsx'


In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor

# Load data
df = pd.read_excel("CFL_DS2_FY25Q2_Forecasted_Simulated.xlsx")
df.columns = df.columns.str.strip()

# Forecast sources to blend
forecast_columns_q2 = [
    "Demand Planners Forecast",
    "Marketing Teams Forecast",
    "Statistical and ML Forecast",
    "FY25 Q2 (Forecasted Mean)"
]

# Check if similar forecasts exist for Q1 (for training)
forecast_columns_q1 = [col.replace("Q2", "Q1") for col in forecast_columns_q2]
if all(col in df.columns for col in forecast_columns_q1) and "FY25 Q1" in df.columns:
    print("Using FY25 Q1 forecast sources to train blending model.")

    X_train = df[forecast_columns_q1]
    y_train = df["FY25 Q1"]

    # Drop rows where any forecast or target is missing
    valid_rows = X_train.notna().all(axis=1) & y_train.notna()
    X_train = X_train[valid_rows]
    y_train = y_train[valid_rows]

    # Train meta-model (Linear or Gradient Boosting)
    meta_model = GradientBoostingRegressor(n_estimators=100, random_state=42)
    meta_model.fit(X_train, y_train)

    # Predict for FY25 Q2 by blending its forecast sources
    X_q2_blend = df[forecast_columns_q2].fillna(0)  # or choose a better imputation
    df["FY25 Q2 (Model-Based Combined Forecast)"] = meta_model.predict(X_q2_blend)

else:
    print("FY25 Q1 forecast source columns not found. Falling back to equal average blending.")
    df["FY25 Q2 (Model-Based Combined Forecast)"] = df[forecast_columns_q2].mean(axis=1)

# Save final forecast
df.to_excel("CFL_DS_FY25Q2_CombinedForecast.xlsx", index=False)
print("📁 Final combined forecast saved as 'CFL_DS_FY25Q2_CombinedForecast.xlsx'")

⚠️ FY25 Q1 forecast source columns not found. Falling back to equal average blending.
📁 Final combined forecast saved as 'CFL_DS_FY25Q2_CombinedForecast.xlsx'
